In [0]:
USE f1_processed;

In [0]:
CREATE TABLE f1_presentation.calculated_race_results
USING parquet
AS
SELECT
      ra.race_year,
      c.name AS team_name,
      d.name AS driver_name,
      re.position,
      re.points,
      11 - re.position AS calculated_points
  FROM f1_processed.results re
  JOIN f1_processed.drivers d on re.driver_id = d.driver_id
  JOIN f1_processed.constructors c on re.constructor_id = c.constructor_id
  JOIN f1_processed.races ra on re.race_id = ra.race_id
WHERE re.position <= 10

In [0]:
SELECT * FROM f1_presentation.calculated_race_results